In [71]:
import json
import pandas as pd
import gzip
import uuid

metadata = "data/meta.json.gz"
reviews = "data/review.json.gz"


def load_data(file_path):
    datas = []
    with gzip.open(file_path, 'rt', encoding='utf-8') as f:
        for line_num, line in enumerate(f):
            try:
                data = json.loads(line.strip())
                data['uuid'] = uuid.uuid4()
                datas.append(data)
            except json.JSONDecodeError as e:
                continue

    df = pd.DataFrame(datas)
    return df

reviews_df = load_data(reviews)
metadata_df = load_data(metadata)

metadata_df.head()
metadata_df.info()

reviews_df.head()
reviews_df.info()

KeyboardInterrupt: 

In [42]:
reviews_df = reviews_df.sample(n=1_000_000, random_state=17398)
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000000 entries, 229523 to 1564777
Data columns (total 9 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   user_id  992759 non-null   object 
 1   name     1000000 non-null  object 
 2   time     1000000 non-null  int64  
 3   rating   992759 non-null   float64
 4   text     555929 non-null   object 
 5   pics     18864 non-null    object 
 6   resp     139051 non-null   object 
 7   gmap_id  1000000 non-null  object 
 8   uuid     1000000 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 76.3+ MB


In [43]:
# Drop duplicated google map reviews, keep first occurrence
duplicate_count = metadata_df.duplicated(subset=['gmap_id']).sum()
print(f"Found {duplicate_count} duplicate entries based on gmap_id.")
metadata_df.drop_duplicates(subset=['gmap_id'], keep='first', inplace=True)

Found 367 duplicate entries based on gmap_id.


In [44]:
metadata_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 74600 entries, 0 to 74966
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              74600 non-null  object 
 1   address           73565 non-null  object 
 2   gmap_id           74600 non-null  object 
 3   description       12207 non-null  object 
 4   latitude          74600 non-null  float64
 5   longitude         74600 non-null  float64
 6   category          74273 non-null  object 
 7   avg_rating        74600 non-null  float64
 8   num_of_reviews    74600 non-null  int64  
 9   price             13603 non-null  object 
 10  hours             55911 non-null  object 
 11  MISC              61249 non-null  object 
 12  state             51510 non-null  object 
 13  relative_results  68431 non-null  object 
 14  url               74600 non-null  object 
 15  uuid              74600 non-null  object 
dtypes: float64(3), int64(1), object(12)
memory us

In [45]:
# Drop useless columns

columns_to_drop = [
    'relative_results', 'url', 'MISC', 'state', 'price',
    'address', 'hours', 'num_of_reviews', 'avg_rating'
]
metadata_df.drop(columns=columns_to_drop, axis=1, inplace=True)

In [46]:
metadata_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 74600 entries, 0 to 74966
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         74600 non-null  object 
 1   gmap_id      74600 non-null  object 
 2   description  12207 non-null  object 
 3   latitude     74600 non-null  float64
 4   longitude    74600 non-null  float64
 5   category     74273 non-null  object 
 6   uuid         74600 non-null  object 
dtypes: float64(2), object(5)
memory usage: 4.6+ MB


In [47]:
import reverse_geocoder as rg

coords = list(zip(metadata_df['latitude'], metadata_df['longitude']))

location_info = rg.search(coords)

states = [loc.get('admin1') for loc in location_info]
countries = [loc.get('cc') for loc in location_info]

metadata_df['state'] = states
metadata_df['country_code'] = countries

metadata_df[['name', 'latitude', 'longitude', 'state', 'country_code']].head()

name   latitude  longitude    state country_code
0               Purple Peanut  34.713163 -86.574041  Alabama           US
1      Posh Mommy & Baby Too!  34.753758 -86.739241  Alabama           US
2  Soccer Shots North Alabama  34.651859 -86.765534  Alabama           US
3          Perdido Pass Jetty  30.273880 -87.559433  Alabama           US
4      Tri-Rx Pharmaceuticals  34.791450 -86.532249  Alabama           US

In [48]:
metadata_df[["category"]].head()

category
0                                [Boutique]
1  [Baby store, Children's furniture store]
2      [Soccer club, Service establishment]
3                            [Fishing pier]
4                        [Corporate office]

In [49]:
# Filter for rows where 'category' is a list with zero items
empty_category_df = metadata_df[metadata_df['category'].apply(lambda x: isinstance(x, list) and len(x) == 0)]

print(empty_category_df.head())

Empty DataFrame
Columns: [name, gmap_id, description, latitude, longitude, category, uuid, state, country_code]
Index: []


In [50]:
# reviews_df drop useless columns, and rename

columns_to_rename = {
    'name': 'reviewer_name',
    'text': 'review',
    'time': 'review_time'
}

reviews_df.drop(columns=['resp'], inplace=True)

reviews_df.rename(columns=columns_to_rename, inplace=True)

reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000000 entries, 229523 to 1564777
Data columns (total 8 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   user_id        992759 non-null   object 
 1   reviewer_name  1000000 non-null  object 
 2   review_time    1000000 non-null  int64  
 3   rating         992759 non-null   float64
 4   review         555929 non-null   object 
 5   pics           18864 non-null    object 
 6   gmap_id        1000000 non-null  object 
 7   uuid           1000000 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 68.7+ MB


In [51]:
# Define the dictionary for renaming
columns_to_rename = {
    'name': 'business_name',
    'description': 'business_description',
    'uuid': 'business_uuid'
}

# Rename the columns in the DataFrame
metadata_df.rename(columns=columns_to_rename, inplace=True)

metadata_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 74600 entries, 0 to 74966
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   business_name         74600 non-null  object 
 1   gmap_id               74600 non-null  object 
 2   business_description  12207 non-null  object 
 3   latitude              74600 non-null  float64
 4   longitude             74600 non-null  float64
 5   category              74273 non-null  object 
 6   business_uuid         74600 non-null  object 
 7   state                 74600 non-null  object 
 8   country_code          74600 non-null  object 
dtypes: float64(2), object(7)
memory usage: 5.7+ MB


In [52]:
# I want see what pics have
reviews_df['pics'].head(10)
# Count null values to see how many reviews lack pictures
null_pics_count = reviews_df['pics'].isnull().sum()
total_rows = len(reviews_df)
print(f"Rows with null 'pics' values: {null_pics_count} ({null_pics_count/total_rows:.2%})")

# Isolate the non-empty entries and show the first one
non_null_pics = reviews_df['pics'].dropna()
if not non_null_pics.empty:
    print("\nExample of a non-null 'pics' entry:")
    print(non_null_pics.iloc[0])
else:
    print("\nNo non-null 'pics' entries found.")

Rows with null 'pics' values: 981136 (98.11%)

Example of a non-null 'pics' entry:
[{'url': ['https://lh5.googleusercontent.com/p/AF1QipMzl5R-6HybpwrQtMSB5fiicMiBzNJO3aiWe1O8=w150-h150-k-no-p']}, {'url': ['https://lh5.googleusercontent.com/p/AF1QipP1HJuivCuCTDGhFdpnCI63sdiELqsxdLZxk8Qh=w150-h150-k-no-p']}, {'url': ['https://lh5.googleusercontent.com/p/AF1QipPfoyZRhgp91SoOD5oCxnreRYqji2Wa0HXZHlmo=w150-h150-k-no-p']}]


In [53]:
def parse_image_urls(pics_data):
    """
    Parses the nested list/dict structure in the 'pics' column
    to extract a simple list of URL strings.
    """
    if not isinstance(pics_data, list):
        return []

    image_urls = []
    for item in pics_data:
        if isinstance(item, dict) and 'url' in item:
            if isinstance(item['url'], list):
                image_urls.extend(item['url'])

    return image_urls

reviews_df['has_image'] = reviews_df['pics'].notna()
reviews_df['images'] = reviews_df['pics'].apply(parse_image_urls)
image_counts = reviews_df['images'].str.len()

multi_pic_count = (image_counts > 1).sum()
print(f"Number of reviews with more than 1 picture: {multi_pic_count}")
reviews_df[reviews_df['has_image']][['pics', 'has_image', 'images']].head()

Number of reviews with more than 1 picture: 9370


pics  has_image  \
4711042  [{'url': ['https://lh5.googleusercontent.com/p...       True   
4887064  [{'url': ['https://lh5.googleusercontent.com/p...       True   
1027376  [{'url': ['https://lh5.googleusercontent.com/p...       True   
2641117  [{'url': ['https://lh5.googleusercontent.com/p...       True   
8336092  [{'url': ['https://lh5.googleusercontent.com/p...       True   

                                                    images  
4711042  [https://lh5.googleusercontent.com/p/AF1QipMzl...  
4887064  [https://lh5.googleusercontent.com/p/AF1QipNTW...  
1027376  [https://lh5.googleusercontent.com/p/AF1QipMeY...  
2641117  [https://lh5.googleusercontent.com/p/AF1QipOJp...  
8336092  [https://lh5.googleusercontent.com/p/AF1QipNv8...

In [54]:
reviews_df.drop(columns=['pics'], inplace=True)
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000000 entries, 229523 to 1564777
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   user_id        992759 non-null   object 
 1   reviewer_name  1000000 non-null  object 
 2   review_time    1000000 non-null  int64  
 3   rating         992759 non-null   float64
 4   review         555929 non-null   object 
 5   gmap_id        1000000 non-null  object 
 6   uuid           1000000 non-null  object 
 7   has_image      1000000 non-null  bool   
 8   images         1000000 non-null  object 
dtypes: bool(1), float64(1), int64(1), object(6)
memory usage: 69.6+ MB


In [55]:
# Check rating null

print(f"Null values in 'rating': {reviews_df['rating'].isnull().sum()}")
print("\nRating column stats:")
reviews_df['rating'].describe()

Null values in 'rating': 7241

Rating column stats:


count    992759.000000
mean          4.242658
std           1.204416
min           1.000000
25%           4.000000
50%           5.000000
75%           5.000000
max           5.000000
Name: rating, dtype: float64

In [56]:
# just want to see what I missing out if i drop all null
print("A look at the original reviews for rows with null ratings:")
print(reviews_df[reviews_df['rating'].isnull()][['reviewer_name', 'review']].head())

A look at the original reviews for rows with null ratings:
            reviewer_name                                             review
8330246            SandyG  The location and staff were amazing.  The hote...
5538869         Anonymous  Not what I expected . I stay at the quality ar...
2966547          Vernessa  I always opted for the more expensive hotel wh...
3156088       Lovtotravel         Please see all the comments on the survey.
3340133  Expedia reviewer  The rooms are huge. Hotel is very pet friendly...


In [57]:
reviews_df['rating'] = reviews_df['rating'].fillna(0)


null_count = reviews_df['rating'].isnull().sum()
print(f"Null values in 'rating' after filling: {null_count}")

reviews_df['rating'] = reviews_df['rating'].astype(int)

print("\nData types after conversion:")
print(reviews_df.dtypes)

Null values in 'rating' after filling: 0

Data types after conversion:
user_id          object
reviewer_name    object
review_time       int64
rating            int64
review           object
gmap_id          object
uuid             object
has_image          bool
images           object
dtype: object


In [58]:
import demoji
import swifter
demoji.download_codes()

def clean_review_text(text):
    # Handle non-string data (like NaN values)
    if not isinstance(text, str):
        return ""

    # Replace emojis with text representation
    text = demoji.replace_with_desc(text)

    # Remove extra whitespace
    text = " ".join(text.split())

    return text

reviews_df['review_cleaned'] = reviews_df['review'].swifter.apply(clean_review_text)
reviews_df[['review', 'review_cleaned']].head(10)

Pandas Apply:   0%|          | 0/1000000 [00:00<?, ?it/s]

review  \
229523                                                None   
8467473                                               None   
7979146  YUM!!!! Definitely beat the twice-as-expensive...   
7747805  Room was not cleaned very well. Found skittles...   
7027868                                               None   
7733117  (Translated by Google) The most popular steakh...   
4813344                                               None   
4711042                Lots of history from army aviation.   
4393377  Associate did a great job of helping me find t...   
3838819  Very clean and the service was friendly and po...   

                                            review_cleaned  
229523                                                      
8467473                                                     
7979146  YUM!!!! Definitely beat the twice-as-expensive...  
7747805  Room was not cleaned very well. Found skittles...  
7027868                                                     
7733117  (Translated by Google) The most popular steakh...  
4813344                                                     
4711042                Lots of history from army aviation.  
4393377  Associate did a great job of helping me find t...  
3838819  Very clean and the service was friendly and po...

In [59]:
reviews_df.drop(columns=["review", "reviewer_name", "user_id"], inplace=True)
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000000 entries, 229523 to 1564777
Data columns (total 7 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   review_time     1000000 non-null  int64 
 1   rating          1000000 non-null  int64 
 2   gmap_id         1000000 non-null  object
 3   uuid            1000000 non-null  object
 4   has_image       1000000 non-null  bool  
 5   images          1000000 non-null  object
 6   review_cleaned  1000000 non-null  object
dtypes: bool(1), int64(2), object(4)
memory usage: 86.6+ MB


In [60]:
metadata_df.drop(columns=["business_uuid", "longitude", "latitude", "business_name"], inplace=True)
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000000 entries, 229523 to 1564777
Data columns (total 7 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   review_time     1000000 non-null  int64 
 1   rating          1000000 non-null  int64 
 2   gmap_id         1000000 non-null  object
 3   uuid            1000000 non-null  object
 4   has_image       1000000 non-null  bool  
 5   images          1000000 non-null  object
 6   review_cleaned  1000000 non-null  object
dtypes: bool(1), int64(2), object(4)
memory usage: 86.6+ MB


In [61]:
merged_df = pd.merge(reviews_df, metadata_df, on='gmap_id', how='left')
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 11 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   review_time           1000000 non-null  int64 
 1   rating                1000000 non-null  int64 
 2   gmap_id               1000000 non-null  object
 3   uuid                  1000000 non-null  object
 4   has_image             1000000 non-null  bool  
 5   images                1000000 non-null  object
 6   review_cleaned        1000000 non-null  object
 7   business_description  513735 non-null   object
 8   category              999763 non-null   object
 9   state                 1000000 non-null  object
 10  country_code          1000000 non-null  object
dtypes: bool(1), int64(2), object(8)
memory usage: 77.2+ MB


In [63]:
merged_df.drop(columns=["gmap_id", "state", "country_code"], inplace=True)

In [66]:
# Convert UUID to string, for better stor
merged_df['uuid'] = merged_df['uuid'].astype(str)

merged_df.to_parquet("cleaned_data.parquet", engine="pyarrow", index=False)

In [67]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 8 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   review_time           1000000 non-null  int64 
 1   rating                1000000 non-null  int64 
 2   uuid                  1000000 non-null  object
 3   has_image             1000000 non-null  bool  
 4   images                1000000 non-null  object
 5   review_cleaned        1000000 non-null  object
 6   business_description  513735 non-null   object
 7   category              999763 non-null   object
dtypes: bool(1), int64(2), object(5)
memory usage: 54.4+ MB
